In [ ]:
import pandas as pd
import os
import numpy as np
import requests
import json
import time
import bs4
from bs4 import BeautifulSoup
headers = {"User-Agent":"Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)"}
headers2 = {"User-Agent":"Mozilla/5.0 AppleWebKit/537.36 (KHTML, like Gecko; compatible; Googlebot/2.1; +http://www.google.com/bot.html) Chrome/41.0.2272.96 Safari/537.36"}

In [ ]:
### in this function we take each car URL and fetching the features we needed
def singleCarDict(url):
    dictionary = {}
    soup_cars = BeautifulSoup(requests.get(url, headers=headers).content)
    dictionary['Region'] = url.split(".")[0].split("/")[2]
    car_price = soup_cars.find("span", class_="price")
    if car_price is not None:
        dictionary['Price'] = car_price.text.replace(',', '').replace('$', '')
    year_n_name = soup_cars.find("p", class_="attrgroup").span.get_text()
    title = year_n_name.split(" ", maxsplit=1)
    if (title[0]).isnumeric:
        dictionary['Year'] = title[0]
        dictionary['Model'] = title[1]
    else:
        dictionary['Model'] = year_n_name
    soup_info = BeautifulSoup(requests.get(url, headers=headers).content)
    info = soup_info.find_all("p", class_="attrgroup")
    clean_info = info[1].get_text()
    cut_info = clean_info.split('\n')
    for word in cut_info:
        if word.split(":")[0] == 'condition':
            dictionary['Condition'] = word.split(":")[1]
        if word.split(":")[0] == 'odometer':
            dictionary['Odometer'] = word.split(":")[1]
        if word.split(":")[0] == 'fuel':
            dictionary['Fuel type'] = word.split(":")[1]
        if word.split(":")[0] == 'transmission':
            dictionary['Transmission'] = word.split(":")[1]
        if word.split(":")[0] == 'paint color':
            dictionary['Paint color'] = word.split(":")[1]
        else:
            continue
    print(dictionary)
    return dictionary

### in this function we take each region URL and making a list of cars URLs. at the end were making a dataframe for each
### region in case we will have a connection error.
def regionDF(reg_url):
    region_df = pd.DataFrame(columns=['Region', 'Model', 'Year', 'Price', 'Odometer', 'Condition', 'Fuel type', 'Transmission', 'Paint color'])
    #a dummy df for efficiency - after every loop of the for loop we merging the dummy df with the main df
    url = reg_url + 'd/cars-trucks/search/cta' #chaining the urls with the search cars and trucks url for efficiency
    soup_cars_trucks = BeautifulSoup(requests.get(url, headers=headers).content) #making a bs object of the data we feched 
    button_next = {'href':'/d/cars-trucks/search/cta'}
    while button_next is not None:
        url = region_url + button_next['href']
        soup_cars_trucks = BeautifulSoup(requests.get(url, headers=headers).content) #making a bs object of the data we feched
        car_urls = [car_url.get("href") for car_url in soup_cars_trucks.find_all("a", class_="result-title")]
        #a for loop to find every single car url
        arr=[]
        for single_car_url in car_urls:
            try:
                arr.append(singleCarDict(single_car_url))
                time.sleep(3)
            except Exception:
                pass
        region_df = region_df.append(arr, ignore_index=True)
        button_next = soup_cars_trucks.find("a", class_="button next") #making an object for the next button
#         break
    region_df.to_csv(region_df['Region'][0] + '.csv', index=False)
    return region_df

In [ ]:
url = 'https://www.craigslist.org/about/sites' #our base url
page = requests.get(url, headers=headers)  #getting the url data
page_soup = BeautifulSoup(page.content, 'html.parser') #making a bs object of the data we feched
url_soup = page_soup.find("div", class_="colmask") #finding all the regions urls
url_list=[u['href'] for u in url_soup.find_all("a", href=True)] #making a list of all the region urls
main_df = pd.DataFrame(columns=['Region', 'Model', 'Year', 'Price', 'Odometer', 'Condition', 'Fuel type', 'Transmission', 'Paint color']) #our main df
for region_url in url_list: #for loop for taking all the cars from all the urls
    main_df = main_df.append(regionDF(region_url))
#     break
main_df.to_csv('CarsDF.csv', index=False)